In [2]:
import os
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Lung-Colon-Cancer-Histopathological-Images-Classification-Project/research'

In [3]:
os.chdir('../')

In [1]:
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Lung-Colon-Cancer-Histopathological-Images-Classification-Project/research'

In [ ]:
os.eviron["MLFLOW_TRACKING_URI"]="https://dagshub.com/Towet-Tum/Lung-Colon-Cancer-Histopathological-Images-Classification-Project.mlflow "
os.environ["MLFLOW_TRACKING_USERNAME"]="Towet-Tum"
os.environ["MLFLOW_TRACKING_PASSWORD"]="39c66009c3d7d6d659cbc691e65a93c46873d3c4"


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    dataset: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
from LungColonClassifier.constants import *
from LungColonClassifier.utils.common import read_yaml, create_directories, save_json

2024-02-02 08:41:40.592771: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 08:41:40.985543: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 08:41:40.985789: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 08:41:41.083378: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 08:41:41.145110: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 08:41:44.211825: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        dataset = os.path.join("artifacts", "data_ingestion", "lung_colon_image_set")
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/lung.model.Xception.h5",
            mlflow_uri="https://dagshub.com/Towet-Tum/Lung-Colon-Cancer-Histopathological-Images-Classification-Project.mlflow ",
            all_params=self.params,
            params_image_size=self.params.IMG_SZ,
            params_batch_size=self.params.BATCH_SIZE,
            dataset=Path(dataset)
        )
        return eval_config


In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from LungColonClassifier.utils.common import load_data, process_data

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config



    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        df = load_data(self.config.dataset)
        _, _, test_gen = process_data(df)
        self.model = self.load_model(self.config.path_of_model)
        ts_length = len(test_gen)
        test_batch_size = min(32, ts_length)
        test_steps = ts_length // test_batch_size
        self.score = self.model.evaluate(test_gen, steps=test_steps, verbose=1)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="XceptionModel")
            else:
                mlflow.keras.log_model(self.model, "model")

In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-02-02 08:43:00,934: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-02 08:43:01,119: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-02 08:43:01,125: INFO: common: created directory at: artifacts]
Found 20000 validated image filenames belonging to 5 classes.
Found 2500 validated image filenames belonging to 5 classes.
Found 2500 validated image filenames belonging to 5 classes.


2024-02-02 08:43:13.244369: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 50466816 exceeds 10% of free system memory.
2024-02-02 08:43:15.006186: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 97329152 exceeds 10% of free system memory.
2024-02-02 08:43:15.486287: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 49561600 exceeds 10% of free system memory.
2024-02-02 08:43:15.487128: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 97329152 exceeds 10% of free system memory.
2024-02-02 08:43:15.690976: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 194658304 exceeds 10% of free system memory.


2/2 [==============================] - 23s 9s/step - loss: 4.6497e-05 - accuracy: 1.0000
[2024-02-02 08:43:53,980: INFO: common: json file saved at: scores.json]


2024/02/02 08:44:01 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-02-02 08:44:21,600: INFO: builder_impl: Assets written to: /tmp/tmp007myrj5/model/data/model/assets]


Successfully registered model 'XceptionModel'.
2024/02/02 08:46:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XceptionModel, version 1
Created version '1' of model 'XceptionModel'.
